In [1]:
import os
import json
from array import array
from csv import writer
from datetime import datetime
from kafka import KafkaProducer
# pip3 install paho-mqtt
import paho.mqtt.publish as publish

%run tweets_consumer.ipynb import TweetConsumer 
%run sentiment_analysis.ipynb import SentimentAnalysis
%run database_service.ipynb import DatabaseService
%run mqtt_broker.ipynb import MQTTProducer

# Sentiment analysis producer with kafka
class SentimentAnalysisProducer(producer="mqtt"):

    if(producer="kafka"):
        producer = KafkaProducer(bootstrap_servers='localhost:9092')

     # docs: https://pypi.org/project/paho-mqtt
    if(producer="mqtt"):
        mqtt = MQTTProducer(broker='localhost',port=1883, topic='tweet_sentiment')
        producer = mqtt.connect_mqtt()

    consumer = TweetConsumer.consumer
    sentiment_analysis = SentimentAnalysis()
    database_service = DatabaseService()
        
    def stream(self):
        # Prepocess the tweets:
        for message in self.consumer:
            tweet = message.value.decode()
            sentiment, polarity, text = self.sentiment_analysis.get_sentiment(tweet)
            sentiment_package = str([sentiment, polarity, text]).encode()
            
            # Send to cold layer
            self.database_service.to_csv(text, polarity, sentiment)
            
            # Send to hot layer
            if(producer="kafka"):
                self.producer.send("tweet_sentiment", sentiment_package)
            
            if(producer="mqtt"):
                self.producer.publish("tweet_sentiment", sentiment_package)
                
            print(sentiment, polarity, text)
            
sentiment_producer = SentimentAnalysisProducer()
sentiment_producer.stream()

SyntaxError: invalid syntax (3289085380.py, line 16)